In [14]:
import numpy as np
import pandas as pd
import string
import os
from collections import Counter

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
pd.options.display.max_colwidth = 50
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

# Creating base

In [15]:
train = pd.read_csv('..//bases/training_variants')
test = pd.read_csv('..//bases/test_variants')

In [16]:
train_texts = pd.read_csv('..//bases/training_text', sep="\|\|", engine='python', header=None, skiprows=1, names=["ID","Text"], encoding = "utf-8")
test_texts = pd.read_csv('..//bases/test_text', sep="\|\|", engine='python', header=None, skiprows=1, names=["ID","Text"], encoding = "utf-8")

In [17]:
train = pd.merge(train, train_texts, how='left', on='ID')
test = pd.merge(test, test_texts, how='left', on='ID')

In [18]:
#Transform Amino Acid (AA) Letter to their three-letter abbreviation in order to find them in the text when they appear
One_to_Three_AA = {'C': 'Cys', 'D': 'Asp', 'S': 'Ser', 'Q': 'Gln', 'K': 'Lys',
         'I': 'Ile', 'P': 'Pro', 'T': 'Thr', 'F': 'Phe', 'N': 'Asn', 
         'G': 'Gly', 'H': 'His', 'L': 'Leu', 'R': 'Arg', 'W': 'Trp', 
         'A': 'Ala', 'V': 'Val', 'E': 'Glu', 'Y': 'Tyr', 'M': 'Met'}
pattern = re.compile('|'.join(One_to_Three_AA.keys()))

In [153]:
#### process the train and test set together
data_all = pd.concat((train, test), axis=0, ignore_index=True)
data_all_backup = data_all[:] ## We keep backup in case we need to use again


# Adding extra feature(s)

In [154]:
# Feature for the length of the text
data_all["Text_words"] = data_all["Text"].map(lambda x: len(str(x).split(" ")))

# Substitutions (subs)
## Functions for pre-processing of subs

In [184]:
# find_sub return the substituions that are in text and those that are not
def find_sub(data):    
    Boolean = [data.Variation[i][:-1] in data.Text[i] or #case 1.
               pattern.sub(lambda x: One_to_Three_AA[x.group()], data.Variation[i][:-1]) # case2
               in data.Text[i]  for i in data.index] ## because new indexing we use 
    
    sub_in_text = data[Boolean]
    not_Boolean = [not i for i in Boolean]  
    sub_not_in_text = data[not_Boolean]
    
    return sub_in_text, sub_not_in_text

# find_sub_numberChange searches for other number of a substitution i.e. G12V -> G_V because sometimes mistake in entry
# Is currently without One_to_three substitution or Variation[:-1] only the full variation
def find_sub_numberChange(data):
    Booleans = [] #will contain the different Booleans if found in text
    for i in data.index:
        split_variation = re.split('(\d+)', data.Variation[i]) # split based on a number
        first_Amino = re.escape(split_variation[0]) #re.escpae uses variable as regex
        last_Amino = re.escape(split_variation[-1])
        new_regex  = first_Amino + r"\d+" + last_Amino
        Boolean = bool(re.search(new_regex, data.Text[i]))
        Booleans.append(Boolean)
        
    sub_number_in_text = data[Booleans]
    not_Boolean = [not i for i in Booleans]  
    sub_number_no_text = data[not_Boolean]
    
    return sub_number_in_text, sub_number_no_text

# for substitutions that are still not found, use other keywords
def find_sub_pattern(data, pattern):    
    Boolean = [pattern in data.Text[i] for i in data.index] ## because new indexing we use 
    
    sub_in_text = data[Boolean]
    not_Boolean = [not i for i in Boolean]  
    sub_not_in_text = data[not_Boolean]
    
    return sub_in_text, sub_not_in_text


In [204]:
##### get_sentences_sub use a window to extract sentences where the subs appear. 
# If window_left & window_right = 0 => just taking the sentences with subs

def get_sentences_sub(data, splitted_sentences, window_left, window_right):
    data.index = range(len(data)) #makes sure that index is right
    sentences_with_sub = [[] for _ in range(len(data))]
    for i in range(len(splitted_sentences)):
        sentences = splitted_sentences[i]
        one_to_three_variation = pattern.sub(lambda x: One_to_Three_AA[x.group()], data.Variation[i][:-1])
        Variation = data.Variation[i][:-1]        
        for j in range(len(sentences)):                              
            if (Variation in sentences[j]) or (one_to_three_variation in sentences[j]):
                new_regex = r"[\S]*" + re.escape(Variation) + r"[\S]*" ###  r"[\S]*" because we look for Variation[:-1] not just Variation
                sentences[j] = re.sub(new_regex, 'placeholderMutation', sentences[j]) #case 1
                ### We add the space to ' placeholderMutation' because sometimes there are letters in front of it
                new_regex = re.escape(one_to_three_variation) + r"[\S]*"
                sentences[j] = re.sub(new_regex, ' placeholderMutation', sentences[j]) #case 2
                sentences_with_sub[i].extend(sentences[max(j-window_left,0) : min(j+1+window_right, len(sentences)-1)])               
    
    return sentences_with_sub

##### get_sentences_sub_number use a window to extract sentences where the subs appear that have different number i.e. G12V -> G_V

def get_sentences_sub_number(data, splitted_sentences, window_left, window_right):
    #position_sentences = [[] for _ in range(len(data))]  #### currently not used
    data.index = range(len(data))
    sentences_with_sub_number = [[] for _ in range(len(data))]
    for i in range(len(splitted_sentences)):
        sentences = splitted_sentences[i] 
        for j in range(len(sentences)):
            split_variation = re.split('(\d+)', data.Variation[i]) # split based on a number
            first_Amino = re.escape(split_variation[0]) #re.escpae uses variable as regex
            last_Amino = re.escape(split_variation[-1])
            new_regex  = first_Amino + r"\d+" + last_Amino
            Boolean = bool(re.search(new_regex, sentences[j]))            
            if Boolean:
                sentences[j] = re.sub(new_regex, ' placeholderMutation', sentences[j]) # Again replacing the sentence with placeholder
                sentences_with_sub_number[i].extend(sentences[max(j-window_left,0) : min(j+1+window_right,len(sentences)-1)])
    
    return sentences_with_sub_number

# for substitutions that are still not found, use other keywords

def get_sentences_pattern(data, splitted_sentences, pattern, window_left, window_right):
    data.index = range(len(data)) #makes sure that index is right
    sentences_with_sub = [[] for _ in range(len(data))]
    for i in range(len(splitted_sentences)):
        sentences = splitted_sentences[i] 
        for j in range(len(sentences)):                              
            if (pattern in sentences[j]):
                sentences_with_sub[i].extend(sentences[max(j-window_left,0) : min(j+1+window_right, len(sentences)-1)])               
    
    return sentences_with_sub

In [157]:
#### Converts list of sentences into one string of sentences for each document => to use for tfidf etc.
def sentences_to_string(sentences_list):
    sentence_strings = []
    for sentences in sentences_list:
        sentence_string =  ' '.join(str(sentence) for sentence in sentences)
        sentence_strings.append(sentence_string)
    
    return sentence_strings 

## Subs processing of the data set 

In [158]:
######### First find those that have the format of being a substitution in data
data_all['Substitutions_var'] = data_all.Variation.apply(lambda x: bool(re.search('^[A-Z]\\d+[A-Z*]$', x))*1) #multiplying by 1 converts True to 1, False to 0 => Maybe modify this later?
data_all['Stop_codon_var'] = data_all.Variation.apply(lambda x: bool(re.search('[*]', x))*1) #multiplying by 1 converts True to 1, False to 0
data_sub = data_all[data_all['Substitutions_var']==1] ### Now we know the index of where a substitution occurs - the data_sub

#### Subs inside the text

In [141]:
## First consider the subs that appear in text
sub_text, sub_no_text = find_sub(data_sub) 

In [142]:
## use tokenizer to split into sentences of all the subs in text 
NLTK_sub = [sent_tokenize(sub_text.Text[i]) for i in sub_text.index] 

In [143]:
# extract window for the sub sentences where they appear
# !! Use [:] because it makes a copy and doesn't change anything to the original indexes
sub_sentences = get_sentences_sub(sub_text[:], NLTK_sub[:], window_left = 0, window_right = 0) # choosing for window 0 as default now
sub_sentences_string = sentences_to_string(sub_sentences)

In [182]:
# Replace text in data_all
data_all.Text.iloc[sub_text.index] = sub_sentences_string #iloc for indexing based on integers
data_all.Text.iloc[sub_text.index]

C:\Users\rafiz\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


1       Using select c-CBL somatic mutations such as S...
2       Using select c-CBL somatic mutations such as S...
3       Most of the changes were novel, although 4 cas...
4       Finally, the third group constituted mutations...
5       Finally, the third group constituted mutations...
6       The second group of mutants (M374V, placeholde...
8       We investigated the mechanism by which placeho...
9       We investigated the mechanism by which CBL-Y37...
10      Finally, the third group constituted mutations...
11      Purified PCR amplicons (CBLWT, CBLC381A, place...
12      Purified PCR amplicons (CBLWT, CBLC381A, CBLK3...
13      Purified PCR amplicons (CBLWT, placeholderMuta...
14      The second group of mutants (M374V, V430M, pla...
15      Purified PCR amplicons (CBLWT, CBLC381A, CBLK3...
17      When introduced into Lin- Sca1+ c-Kit+ (LSK) H...
18      The second group of mutants placeholderMutatio...
19      When introduced into Lin- Sca1+ c-Kit+ (LSK) H...
20      Using 

#### Subs with a different number inside the text


In [174]:
# now the subs that don't appear in text: one reason is different number in the substitution. f.e. G12V -> G13V 
sub_number_text, sub_number_no_text = find_sub_numberChange(sub_no_text) # 108 such cases out of 411 = nice improvement

In [175]:
## use tokenizer to split into sentences of subs that have different number in text 
NLTK_sub_number = [sent_tokenize(sub_number_text.Text[i]) for i in sub_number_text.index]

In [176]:
# extract window for the sub sentences where they appear
# !! Use [:] because it makes a copy and doesn't change anything to the original indexes
sub_number_sentences = get_sentences_sub_number(sub_number_text[:], NLTK_sub_number[:], window_left = 0, window_right = 0)
sub_number_string = sentences_to_string(sub_number_sentences)

In [181]:
data_all.Text.iloc[sub_number_text.index] = sub_number_string #iloc for indexing based on integers
data_all.Text.iloc[sub_number_text.index]

C:\Users\rafiz\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


44      His-tagged versions of the catalytic region of...
49      His-tagged versions of the catalytic region of...
61      His-tagged versions of the catalytic region of...
63      His-tagged versions of the catalytic region of...
66      His-tagged versions of the catalytic region of...
73      While both were missense mutations, in silico ...
208      placeholderMutation and T847I were also found...
213     Complex mutation  placeholderMutation and dele...
267     As a positive control, we introduced the well-...
275     And in the series of Pallis and colleagues (12...
306     Recurrent mutations in H3F3A, which encodes th...
307     Histone H3-K42me2 may regulate transcription  ...
308     Recurrent mutations in H3F3A, which encodes th...
521     Both combinations contained mutations of codon...
607     In the present study, we have focused on two d...
632     Such mutant amino acid specificity was also ap...
730     In contrast, ectopic expression of FGFR4  plac...
1081    Recent

#### Subs that are still not found

In [200]:
# subs that are still not found, look based on a pattern like the word 'mutat' in text
sub_pattern_text, sub_pattern_no_text = find_sub_pattern(sub_number_no_text, 'mutat') # 108 such cases out of 411 = nice improvement

In [201]:
sub_pattern_no_text # only 14 texts that include sentences with the word 'mutat'

,Class,Gene,ID,Text,Variation,Substitutions_var,Stop_codon_var
140,5.0,EGFR,140,The accurate determination of perfluoroalkyl s...,I491M,1,0
145,2.0,EGFR,145,The accurate determination of perfluoroalkyl s...,K467T,1,0
259,2.0,EGFR,259,The accurate determination of perfluoroalkyl s...,S464L,1,0
416,4.0,TP53,416,The effects of chlorpromazine on various prope...,P151S,1,0
1109,1.0,FANCA,1109,null,S1088F,1,0
1407,6.0,FGFR3,1407,null,K508M,1,0
1613,4.0,VHL,1613,The case of a 40-year-old woman with severe ed...,L158Q,1,0
1936,7.0,CARD11,1936,Left ventricular (LV) remodeling is a signific...,G116S,1,0
1942,7.0,CARD11,1942,Left ventricular (LV) remodeling is a signific...,E127G,1,0
2302,7.0,JAK1,2302,Regulatory T (T reg) cells have a major role i...,R724H,1,0


In [202]:
## use tokenizer to split into sentences of subs that have keyword 'mutat'
NLTK_sub_pattern = [sent_tokenize(sub_pattern_text.Text[i]) for i in sub_pattern_text.index]

In [206]:
# !! Use [:] because it makes a copy and doesn't change anything to the original indexes
sub_pattern_sentences = get_sentences_pattern(sub_pattern_text[:], NLTK_sub_pattern[:], 'mutat', window_left = 0, window_right = 0)
sub_pattern_string = sentences_to_string(sub_pattern_sentences)

In [213]:
data_all.Text.iloc[sub_pattern_text.index] = sub_pattern_string #iloc for indexing based on integers
data_all.Text.iloc[sub_pattern_text.index]

C:\Users\rafiz\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


46      Mutations in CBL, encoding an E3 ubiquitin lig...
80      This D1 domain contains only two serine residu...
102     Identification of a high-risk disease-causing ...
104     In this study, we profiled 70 colorectal cance...
106     Identification of a high-risk disease-causing ...
108     Most Lynch Syndrome-associated CRCs are report...
111     Identification of a high-risk disease-causing ...
112     Germ-line MSH6 mutations, which are rare in HN...
136     In 2011, whole-exome sequencing studies showed...
151     We investigated the association between mutati...
155     Clinical data and epidermal growth factor rece...
161     Clinical data and epidermal growth factor rece...
168     Currently available methods of EGFR mutation d...
178     Those were analyzed for EGFR and k-ras (Kirste...
182     Purpose: Clinical reports about responsiveness...
185     ‘Classical' mutations in the EGFR tyrosine kin...
215     Purpose: Clinical features of epidermal growth...
222     Findin

# Non-substitutions (no subs)
## Functions for pre-processing of no_subs like fusions, deletions, truncations, etc.

In [71]:
def custom_find(data):
    l=["mutat","variat","fusion","deleti","amplific","framesh",
    "truncat","fs","exon","dupl"]
    Boolean={}
    for word in l:
        Boolean[word] = [re.search(word,data.Text[i]) for i in data.index] ## because new indexing we use 
    Bool=[any(tup) for tup in zip(Boolean["mutat"],Boolean["variat"],Boolean["fusion"],
        Boolean["deleti"],Boolean["amplific"],Boolean["truncat"],
        Boolean["fs"],Boolean["framesh"],Boolean["exon"],Boolean["dupl"])]
    no_sub_in_text=data[Bool]
    no_Bool=[not i for i in Bool]
    no_sub_not_in_text=data[no_Bool]
    return no_sub_in_text,no_sub_not_in_text


In [73]:
def custom_find_bis(data):
    l=["mutat","variat"]
    Boolean={}
    for word in l:
        Boolean[word] = [re.search(word,data.Text[i]) for i in data.index] ## because new indexing we use 
    Bool=[any(tup) for tup in zip(Boolean["mutat"],Boolean["variat"])]
    no_sub_in_text=data[Bool]
    no_Bool=[not i for i in Bool]
    no_sub_not_in_text=data[no_Bool]
    return no_sub_in_text,no_sub_not_in_text


In [75]:
def get_sentences_nosub(data, splitted_sentences, window_left, window_right):
    #position_sentences = [[] for _ in range(len(data))]  #### currently not used
    data.index = range(len(data))
    sentences_with_sub = [[] for _ in range(len(data))]
    background=[[] for _ in range(len(data))]
    
    for i in range(len(splitted_sentences)):
        sentences = splitted_sentences[i]
        l=["mutat","variat","fusion","deleti","amplific","framesh",
    "truncat","fs","exon","dupl"]     
        for j in range(len(sentences)):    
            for keyword in l:
                if keyword in sentences[j]:
                    sentences_with_sub[i].extend(sentences[max(j-window_left,0) : min(j+1+window_right, len(sentences)-1)])
                else:
                    background[i].extend(sentences[max(j-window_left,0) : min(j+1+window_right, len(sentences)-1)])
    return sentences_with_sub,background   ### This might take a while because it's looping through all sentences


In [78]:
def get_sentences_sub_again_notext(data, splitted_sentences, window_left, window_right):
    #position_sentences = [[] for _ in range(len(data))]  #### currently not used
    data.index = range(len(data))
    sentences_with_sub = [[] for _ in range(len(data))]
    background=[[] for _ in range(len(data))]
    
    for i in range(len(splitted_sentences)):
        sentences = splitted_sentences[i]
        l=["mutat","variat"]    
        for j in range(len(sentences)):    
            for keyword in l:
                if keyword in sentences[j]:
                    sentences_with_sub[i].extend(sentences[max(j-window_left,0) : min(j+1+window_right, len(sentences)-1)])
                else:    
                    background[i].extend(sentences[max(j-window_left,0) : min(j+1+window_right, len(sentences)-1)])
    return sentences_with_sub,background   ### This might take a while because it's looping through all sentences

## No subs processing of the data set 

In [214]:
#### First find those that have the format of being a non-substitutions in the data

# Initialize some of the variables already (because we splice them, could be filled with NA's otherwise)
# Represents 783 from train and test set
data_all['gene_fusion_var'] = 0
data_all['Deletion_var'] = 0
data_all['del_or_ins_var'] = 0

####### Fusions : 'Fusions' ############
data_all['Fusion_var'] = data_all.Variation.apply(lambda x: bool(re.search('^fusion', x, re.IGNORECASE))*1) #multiplying by 1 converts True to 1, False to 0
_ , new_data_all = variation_regex(data_all, '^fusion')  #37 cases

###### Fusions: 'Gene-Gene fusion' ########
data_all['gene_fusion_var'].loc[new_data_all.index] = new_data_all.Variation.apply(lambda x: bool(re.search('fusion', x, re.IGNORECASE))*1)
_ , new_data_all = variation_regex(new_data_all, 'fusion') #160 cases

####### Deletions: 'Deletions' ############
data_all['Deletion_var'].loc[new_data_all.index] = new_data_all.Variation.apply(lambda x: bool(re.search('^del', x, re.IGNORECASE))*1)
_, new_data_all = variation_regex(new_data_all, '^del') # 88 cases

####### Deletions & Insertions wheteher together or seperately (doesn't make a big difference IMO)
data_all['del_or_ins_var'].loc[new_data_all.index] = new_data_all.Variation.apply(lambda x: bool(re.search('del|ins', x, re.IGNORECASE))*1)
# 196 cases

###### Amplifications #########
data_all['Amplification_var'] = data_all.Variation.apply(lambda x: bool(re.search('ampl', x, re.IGNORECASE))*1) # 79 cases

###### Truncations ########### Don't forget there are 'Truncating mutations'  '_trunc' 
data_all['Truncation_var'] = data_all.Variation.apply(lambda x: bool(re.search('trunc', x, re.IGNORECASE))*1) # 118 cases

####### Exons #########
data_all['exon_var'] = data_all.Variation.apply(lambda x: bool(re.search('exon', x, re.IGNORECASE))*1) 

####### Frameshift mutations ########
data_all['frameshift_var'] = data_all.Variation.apply(lambda x: bool(re.search('fs', x, re.IGNORECASE))*1) # 22 cases

####### Duplications ##############
data_all['dup_var'] = data_all.Variation.apply(lambda x: bool(re.search('dup', x, re.IGNORECASE))*1) 

all_variations = data_all.loc[:, "Substitutions_var":"dup_var"]

C:\Users\rafiz\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [253]:
# Remember there are still unassigned variations: 62 in total
data_all[(all_variations.T == 0).all()]

,Class,Gene,ID,Text,Variation,Text_words,Substitutions_var,Stop_codon_var,gene_fusion_var,Deletion_var,del_or_ins_var,Fusion_var,Amplification_var,Truncation_var,exon_var,frameshift_var,dup_var
31,7.0,TERT,31,Sequencing studies have identified many recurr...,Promoter Mutations,21096,0,0,0,0,0,0,0,0,0,0,0
48,4.0,PTPRT,48,Metastasis is the leading cause of death for g...,Promoter Hypermethylation,8565,0,0,0,0,0,0,0,0,0,0,0
87,7.0,CCNE1,87,A variety of results point to the transcriptio...,Overexpression,17829,0,0,0,0,0,0,0,0,0,0,0
150,7.0,EGFR,150,Alterations ofthe EGFR gene occur frequently i...,EGFRvIII,25595,0,0,0,0,0,0,0,0,0,0,0
152,7.0,EGFR,152,Abstract Glioblastomas with EGFR amplificatio...,EGFRvII,12517,0,0,0,0,0,0,0,0,0,0,0
170,7.0,EGFR,170,Abstract The epidermal growth factor recepto...,EGFRvV,9121,0,0,0,0,0,0,0,0,0,0,0
211,7.0,EGFR,211,Oncogenic EGFR mutations are found in 10-35% o...,EGFR-KDD,2724,0,0,0,0,0,0,0,0,0,0,0
238,7.0,EGFR,238,Tumor cells often subvert normal regulatory me...,EGFRvIV,7536,0,0,0,0,0,0,0,0,0,0,0
285,7.0,EIF1AX,285,Poorly differentiated thyroid cancer (PDTC) an...,A113_splice,5573,0,0,0,0,0,0,0,0,0,0,0
350,1.0,CDH1,350,"In diffuse gastric carcinoma, despite common E...",Epigenetic Silencing,3955,0,0,0,0,0,0,0,0,0,0,0


In [76]:
NLTK_nosub = [sent_tokenize(no_sub_yes.Text[i]) for i in no_sub_yes.index]

In [77]:
nosub_sentences,nosub_background = get_sentences_nosub(no_sub_yes, NLTK_nosub, window_left = 0, window_right = 0) # Retrieves sentences where keyword is included
nosub_sentences = [sorted(set(sentences), key = sentences.index) for sentences in nosub_sentences] # only use unique 
nosub_background = [sorted(set(sentences), key = sentences.index) for sentences in nosub_background] # only use unique 



In [79]:
NLTK_sub_again_notext = [sent_tokenize(sub_again_yes.Text[i]) for i in sub_again_yes.index]


In [80]:
sub_again_sentences,sub_again_background = get_sentences_sub_again_notext(sub_again_yes, NLTK_sub_again_notext, window_left = 0, window_right = 0) # Retrieves sentences where keyword is included
sub_again_sentences = [sorted(set(sentences), key = sentences.index) for sentences in sub_again_sentences] # only use unique 
sub_again_background = [sorted(set(sentences), key = sentences.index) for sentences in sub_again_background] # only use unique 


In [81]:
NLTK_sub_noText = [sent_tokenize(sub_noText.Text[i]) for i in sub_noText.index]
sub_noText_sentences,sub_noText_background = get_sentences_sub_noText(sub_noText, NLTK_sub_noText, window_left = 0, window_right = 0) # Retrieves sentences where subsitution mutation is included
sub_noText_sentences = [sorted(set(sentences), key = sentences.index) for sentences in sub_noText_sentences] # only use unique sentences
sub_noText_background = [sorted(set(sentences), key = sentences.index) for sentences in sub_noText_background] # only use unique 


In [82]:
NLTK_sub = [sent_tokenize(sub_in_text.Text[i]) for i in sub_in_text.index] # takes a long time to run tokenizer => use pickle to save
sub_sentences,sub_background = get_sentences_sub(sub_in_text, NLTK_sub, window_left = 0, window_right = 0) 
# Retrieves sentences where subsitution mutation is included.
# window_left and window_right specify which sentences to keep at the left side or right side of the sub sentences.
# IMPORTANT: I used also placeholderMutation to replace the original sub mutations here
sub_sentences = [sorted(set(sentences), key = sentences.index) for sentences in sub_sentences]
sub_background = [sorted(set(sentences), key = sentences.index) for sentences in sub_background]


In [83]:
sub_sentences_string = sentences_to_string(sub_sentences)
sub_noText_string = sentences_to_string(sub_noText_sentences)
nosub_sentences_string=sentences_to_string(nosub_sentences)
sub_again_string=sentences_to_string(sub_again_sentences)

In [84]:
#Creating the background text (Text is now the frontline text)

In [86]:
#data_all.loc[sub_in_text.index,"Background"]=sentences_to_string(sub_background)
#data_all.loc[sub_noText.index,"Background"]=sentences_to_string(sub_noText_background)
#data_all.loc[no_sub_yes.index,"Background"]=sentences_to_string(nosub_background)
#data_all.loc[sub_again_yes.index,"Background"]=sentences_to_string(sub_again_background)
#data_all.loc[sub_again_no.index,"Background"]="nbckgr"
#data_all.loc[no_sub_no.index,"Background"]="nbckgr"

In [ ]:
#data_all.Background[data_all.Background==""]="nbckgr"
#data_all.Background[data_all.Background.isnull()]="nbckgr"

In [89]:
#TODO : #Sentence Tokenizer for non-subs that are not in text, AND
#subs that are not in text

In [90]:
#Not used
################ The dummy variables for Gene and Text ##################
## TODO: also use dummy for Text? There are 135 shared Genes and 142 shared Text between train and Leaks!  len(set(train.Text) & set(Leaks.Text))
#data_all_dummy = data_all_backup[['Gene', 'Text']] # drop those columns we don't need as dummy.
#X_dummy = pd.get_dummies(data_all_dummy) # converts categorical variables into dummy variable. From len set => 269 genes + 2090 texts
#X_dummy_train = X_dummy[:train.shape[0]]
#X_dummy_test = X_dummy[train.shape[0]:]
#dummy_names = X_dummy.columns.values #### To remember names if you want to check again what Gene or Text used
#X_dummy = X_dummy.values

In [91]:
###### Use the variation types 
#variation_types = data_all.drop(['ID', 'Gene', 'Class', 'Text', 'Variation'], axis =1)
#X_variation_train = variation_types[:train.shape[0]]
#X_variation_test = variation_types[train.shape[0]:]
#variation_names = variation_types.columns.values 

# Cleaning 

In [92]:
stop = set(stopwords.words('english'))
exclude = set('!"#$%&\'()*+:;<=>?@[\\]^_`{|}~') 
lemma = WordNetLemmatizer()
def clean(doc,lemmatiz=False):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free_0 = [re.sub(",|\.|/"," ",ch) for ch in stop_free]
    if lemmatiz:
        punc_free_lem="".join(ch for ch in punc_free_0 if ch not in exclude)
        normalized = " ".join(lemma.lemmatize(word) for word in punc_free_lem.split())
        return normalized
    else:
        punc_free = "".join(ch for ch in punc_free_0 if ch not in exclude)
        return punc_free

In [93]:
#No lemmatization for the moment, be careful not to lemmatize then w2vec
data_all.Text = [clean(doc) for doc in data_all.Text]  
#data_all.Background = [clean(doc) for doc in data_all.Background]

AttributeError: 'numpy.float64' object has no attribute 'lower'

# We gonna extract the main info of the background using tfidf+tsvd

In [94]:
ID_train=train.ID
ID_test=test.ID

In [95]:
train = data_all.iloc[:len(train)]
test = data_all.iloc[len(train):]

In [96]:
#tfidf = TfidfVectorizer(
 #       min_df=3, max_features=5000, strip_accents=None, lowercase = False,
  #      analyzer='word', token_pattern=r'\w+', ngram_range=(1,3), use_idf=True,
   #     smooth_idf=True, sublinear_tf=True
    #    ).fit(train["Background"])

In [97]:
#X_train_background = tfidf.transform(train["Background"])
#X_test_background = tfidf.transform(test["Background"])

In [98]:
#tsvd=TruncatedSVD(n_components=100,n_iter=10,random_state=26)
#tsvd_train=tsvd.fit_transform(X_train_background)
#tsvd_test=tsvd.transform(X_test_background)
    

In [99]:
#for i in range(100):
#   train['tsvd_'+str(i)] = tsvd_train[:, i]
#   test['tsvd_'+str(i)] = tsvd_test[:, i]
    

In [100]:
data_all=pd.concat((train, test), axis=0, ignore_index=True)

In [104]:
train_1 = data_all.iloc[:len(train)]
test_1 = data_all.iloc[len(train):]